<a href="https://colab.research.google.com/github/ua-datalab/Geospatial_Workshops/blob/main/notebooks/STAC_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install pystac_client. This library is used to crawl SpatioTemporal Asset Catalogs (STAC)
!pip install pystac_client --quiet
!pip install planetary_computer --quiet

In [16]:
##Install library to work with raster images

! pip install rasterio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 50.5 MB/s eta 0:00:00


In [2]:
#Bring the libraries into the running environment

import pystac_client
import planetary_computer

In [38]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import requests
import numpy as np





In [29]:
import requests

In [8]:
from pystac_client import Client
catalog = Client.open('https://landsatlook.usgs.gov/stac-server/')
catalog.title

collections = list(catalog.get_collections())

# Print the number of collections
print(f"Number of collections in the base catalog: {len(collections)}")


Number of collections in the base catalog: 14


In [10]:
# Print the names (or IDs) and descriptions of each collection
for collection in collections:
    print(f"ID: {collection.id}")
    print(f"Title: {collection.title}")  # Optional: if you want to print the title as well
    print(f"Description: {collection.description}")

ID: landsat-c2l2-sr
Title: Landsat Collection 2 Level-2 UTM Surface Reflectance (SR) Product
Description: The Landsat Surface Reflectance (SR) product measures the fraction of incoming solar radiation that is reflected from Earth's surface to the Landsat sensor.
ID: landsat-c2l2-st
Title: Landsat Collection 2 Level-2 UTM Surface Temperature (ST) Product
Description: The Landsat Surface Temperature (ST) product represents the temperature of the Earth's surface in Kelvin (K).
ID: landsat-c2ard-st
Title: Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Surface Temperature (ST) Product
Description: The Landsat Surface Temperature (ST) product represents the temperature of the Earth's surface in Kelvin (K).
ID: landsat-c2l2alb-bt
Title: Landsat Collection 2 Level-2 Albers Top of Atmosphere Brightness Temperature (BT) Product
Description: The Landsat Top of Atmosphere Brightness Temperature (BT) product is a top of atmosphere product with radiance calculated 'at-sensor', not atmosp

In [37]:
#Do a temporal and geographic box search
time_range = "2020-12-01/2020-12-31"
bbox = [-111.185821, 32.047978, -110.690757, 32.41685]

search = catalog.search(collections=["landsat-c2ard-sr"], bbox=bbox, datetime=time_range)
items = search.item_collection()

# Print the total number of items found
print(f"Total number of items found: {len(items)}")

for item in items:
    print(f"Item ID: {item.id}")

    # Print the asset names (keys) for each item
    print("Assets:")
    for asset_key in item.assets:
        print(f"  - {asset_key}: {item.assets[asset_key].href}")
    print("---------------------------------------------------")

Total number of items found: 7
Item ID: LE07_CU_007014_20201231_20210504_02_SR
Assets:
  - thumbnail: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02_thumb_small.jpeg
  - reduced_resolution_browse: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02_thumb_large.jpeg
  - index: https://landsatlook.usgs.gov/stac-browser/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02
  - blue: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02_SR_B1.TIF
  - green: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02_SR_B2.TIF
  - red: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE

In [45]:
cog_path = 'https://landsatlook.usgs.gov/tile/collection02/oli-tirs/2020/CU/007/014/LC08_CU_007014_20201207_20210504_02/LC08_CU_007014_20201207_20210504_02_SR_B6.TIF'

In [46]:
save_path = '/content/STAC_test.tif'

In [47]:
response = requests.get(cog_path)

In [48]:
if response.status_code == 200:
    # Write the content of the response to a file in the specified save path
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully: {save_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded successfully: /content/STAC_test.tif


In [49]:
## Install libraries for visualization and analysis

!pip install rasterio
!pip install GDAL

## bring the libraries into the environment
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

In [50]:
# Set a variable containing the path of the imagery on your Colab disk
image_path = '/content/STAC_test.tif'  # Replace with your file path

In [53]:
### Visualize the Satellite Imagery

with rasterio.open(image_path) as src:
    image = src.read(1)  # Display as false-color composite (show nir as red; show red as green; show green as blue)
    # Read the first band (assuming it's a single band image)
    band1 = src.read(1)

    # Plot the image
    plt.figure(figsize=(10, 10))
    plt.imshow(band1, cmap='gray')
    plt.title('Landsat Single Band Image')
    plt.colorbar(label='Digital Number (DN) Values')
    plt.show()


RasterioIOError: '/content/STAC_test.tif' not recognized as a supported file format.